# Plot neutralization curves

In this notebook we will create neutralization plots from GFP-based neutralization assays. 
The GFP-based neutralization assay system is described in detail in [Bloom et al (2010)](http://science.sciencemag.org/content/328/5983/1272.long) and [Hooper and Bloom (2013)](https://jvi.asm.org/content/87/23/12531).

In brief, we generate by reverse genetics influenza viruses that carry GFP in the PB1 segment. We then quantified using a plate reader the GFP signal produced by MDCK-SIAT1-CMV-PB1 cells infected with PB1flank-eGFP virus that had been incubated with various dilutions of antibody or serum. The neutralization curves represent the mean and standard deviation of three measurements, with each replicate in a separate column of a 96-well plate.

In [1]:
import os
import warnings

from IPython.display import display, HTML

import numpy

import pandas as pd

from plotnine import *

import yaml

import neutcurve
from neutcurve.colorschemes import CBPALETTE
import neutcurve.parse_excel

print(f"Using neutcurve version {neutcurve.__version__}")

Using neutcurve version 0.1.0


In [2]:
pd.set_option('display.float_format', '{:.3g}'.format)

In [3]:
# Hide warnings that can clutter output
warnings.simplefilter('ignore')

## Configuration and setup
Read general configuration from [config.yaml](config.yaml):

In [16]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

Read the neutralization assay configuration from the specified file:

In [17]:
print(f"Reading neutralization assay setup from {config['neut_config']}")

with open(config['neut_config']) as f:
    neut_config = yaml.safe_load(f)

Reading neutralization assay setup from data/neut_config.yaml


Get the output directory:

In [ ]:
outdir = config['plot_outdir']
os.makedirs(outdir, exist_ok=True)
print(f"Output will be written to {outdir}")

## Read neutralization data

Next, for each dict in _neut_config_, we use `neutcurve.parse_excel.parseRachelStyle2019` to create a tidy data frame appropriate for passing to `neutcurve.CurveFits`. We then concatenate all the tidy data frames to get our neutralization data:

In [15]:
neutdata = []  # store all data frame, then concatenate at end

for sampledict in neut_config:
    assert len(sampledict) == 1
    sampleset, kwargs = list(sampledict.items())[0]
    print(f"Parsing data for {sampleset}...")
    neutdata.append(neutcurve.parse_excel.parseRachelStyle2019(**kwargs))

neutdata = pd.concat(neutdata)
print(f"Read data for {len(neutdata.groupby('serum'))} sera and "
      f"{len(neutdata.groupby(['serum', 'virus']))} serum / virus pairs.")
      
assert neutdata['virus'].unique() == ['wt'], 'expected data only for wildtype virus'

Parsing data for VIDD1...
Parsing data for VIDD2...
Parsing data for VIDD3...
Parsing data for VIDD4...
Parsing data for VIDD5...
Parsing data for 557v1...
Parsing data for 557v2...
Parsing data for 574v1...
Parsing data for 574v2...
Parsing data for 589v2...
Parsing data for 571v2...
Parsing data for ferret-Pitt-1-postinf...
Parsing data for ferret-Pitt-2-postinf...
Parsing data for ferret-Pitt-3-postinf...
Parsing data for ferret-WHO-Victoria2011...


ValueError: `excelfile` data/neut_plate_reader_data/WHOVic11_NeutralizationAssay.xlsx has the following extra sheets not in `sheet_mapping`: {'F159G', 'K160T'}

In [ ]:
mut = ['f9437neg', 'f9437post-infection']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [sigmoid, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(f9437, 'serum_dilution', 
                         mut,
                         './plots/ferret_Pitt_9437.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_Pitt_9437.pdf')

In [ ]:
mut = ['WHOCC']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo_fixed_topbottom]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(fWHOCC_Perth2009, 'serum_dilution', 
                         mut,
                         './plots/ferret_WHOCC_Perth2009.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_WHOCC_Perth2009.pdf')

In [ ]:
mut = ['WHOCCVic']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(fWHOCC_Vic11, 'serum_dilution', 
                         mut,
                         './plots/ferret_WHOCC_Vic11.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_WHOCC_Vic11.pdf')